In [1]:
import json
import gc
import random
import networkx as nx
from networkx.readwrite import json_graph
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
import scml
import mylib

In [2]:
# number of context words in the left and right directions
m = 2
iters = 1
limit = 0
max_choices = 2000
type2id = {
    "click": 0,
    "cart": 1,
    "order": 2,
}

In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(31)

In [4]:
%%time
with open("input/vocab2.json") as f:
    id2label = [w for w, c in json.load(f)]
label2id = {k:v for v, k in enumerate(id2label)}
print(f"len(label2id)={len(label2id):,}")

len(label2id)=1,855,603
Wall time: 2.21 s


In [5]:
del id2label
gc.collect()

21

In [6]:
%%time
with open("input/graph.json") as f:
    g = nx.adjacency_graph(json.load(f))
print(g)

DiGraph with 3747723 nodes and 99004731 edges
Wall time: 8min 6s


In [8]:
rows = []
_nodes = g.nodes
if limit>0:
    _nodes = list(_nodes)[:limit]
out_edges = {}
in_edges = {}
for i in range(iters):
    for center in tqdm(_nodes, desc=f"i={i}"):
        # going left: predecessor nodes
        center_word, center_type = tuple(center.split("_"))
        curr = center
        for _ in range(m):
            if curr not in in_edges:
                choices = []
                for u, v, d in g.in_edges(curr, data=True):
                    if len(choices)>max_choices:
                        break
                    choices.append((u, d["weight"]))
                in_edges[curr] = choices
            choices = in_edges[curr]
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            if curr==center:
                break
            curr_word, curr_type = tuple(curr.split("_"))
            if curr_word!=center_word:
                rows.append({
                    "center_word": label2id[center_word],
                    "center_type": type2id[center_type],
                    "outside_word": label2id[curr_word],
                    "outside_type": type2id[curr_type],
                })
        # going right: successor nodes
        curr = center
        for _ in range(m):
            if curr not in out_edges:
                choices = []
                for u, v, d in g.out_edges(curr, data=True):
                    if len(choices)>max_choices:
                        break
                    choices.append((v, d["weight"]))
                out_edges[curr] = choices
            choices = out_edges[curr]
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            if curr==center:
                break
            curr_word, curr_type = tuple(curr.split("_"))
            if curr_word!=center_word:
                rows.append({
                    "center_word": label2id[center_word],
                    "center_type": type2id[center_type],
                    "outside_word": label2id[curr_word],
                    "outside_type": type2id[curr_type],
                })

i=0: 100%|███████████████████| 3747723/3747723 [19:28<00:00, 3208.04it/s]


In [9]:
df = pd.DataFrame.from_records(rows)
df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
cols = ["center_word", "outside_word"]
df[cols] = df[cols].astype(np.int32)
cols = ["center_type", "outside_type"]
df[cols] = df[cols].astype(np.int8)
df.info()
assert (df["center_word"]!=df["outside_word"]).all()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12651630 entries, 0 to 12651629
Data columns (total 4 columns):
 #   Column        Dtype
---  ------        -----
 0   center_word   int32
 1   center_type   int8 
 2   outside_word  int32
 3   outside_type  int8 
dtypes: int32(2), int8(2)
memory usage: 120.7 MB


In [10]:
df.describe(percentiles=percentiles)

,center_word,center_type,outside_word,outside_type
count,1.265163e+07,1.265163e+07,1.265163e+07,1.265163e+07
mean,8.034785e+05,6.043214e-01,3.757538e+05,2.510347e-01
std,5.211632e+05,7.498212e-01,4.466851e+05,6.022091e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1%,1.364900e+04,0.000000e+00,1.230000e+02,0.000000e+00
5%,6.735100e+04,0.000000e+00,1.709000e+03,0.000000e+00
10%,1.350970e+05,0.000000e+00,6.177000e+03,0.000000e+00
20%,2.748290e+05,0.000000e+00,2.362500e+04,0.000000e+00
30%,4.225960e+05,0.000000e+00,5.508300e+04,0.000000e+00
40%,5.797960e+05,0.000000e+00,1.047540e+05,0.000000e+00


In [11]:
df.head(30)

,center_word,center_type,outside_word,outside_type
0,1,1,35,0
1,35,0,1,0
2,35,0,1,1
3,35,1,997,0
4,7,0,24171,0
5,7,0,26977,0
6,7,1,44,0
7,7,1,634,2
8,3,0,4,0
9,3,0,94,0


In [12]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/pairs.parquet", index=False)

Wall time: 857 ms


In [13]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:30:44.103689
